## File 01 - Exploratory Data Analysis of Interaction-level Data

In this file, we open the CSV in which the interaction-level ecommerce data is stored and we run some Spark SQL queries on it.

### Set up Spark session and data schema

We can specify more options in the SparkSession creator, but currently the options are at the default settings.

In [1]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql import types as T

spark = SparkSession.builder \
        .appName("project") \
        .getOrCreate()

sc = spark.sparkContext

# In the case of needing to specify a schema (recommended behavior), we can use this code to help design it rapidly
#   by using the default Spark settings and then making our own modifications.

# To infer schema without using all data:
# df = spark.read.option("header","true") \
#         .option("samplingRatio",0.001) \
#         .csv("/project/ds5559/group12/raw_data/2019-10.csv")
# To print the basic schema as a DDL:
# print(df._jdf.schema().toDDL())

# We have completed the above step (this is our more accurate schema)
# Defining the schema is unnecessary now that we are reading in parquets. 
#schema = "`event_time` TIMESTAMP,`event_type` STRING,`product_id` INT,`category_id` BIGINT,`category_code` STRING,`brand` STRING,`price` FLOAT,`user_id` INT,`user_session` STRING"
#ddl_schema = T._parse_datatype_string(schema)

CPU times: user 181 ms, sys: 139 ms, total: 321 ms
Wall time: 8.4 s


### Read in dataframe

In [2]:
%%time
df = spark.read.parquet("/project/ds5559/group12/raw_data/Filtered_Original.parquet")

CPU times: user 1.87 ms, sys: 2.08 ms, total: 3.95 ms
Wall time: 2.75 s


### Print number of records in dataframe

N.B.: this takes **>20 s** for perhaps one of the simplest possible operations. So we should be careful to take exploratory actions only on subsets of the data.

In [3]:
%%time
df.printSchema()
print(str(df.count())+" records")

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: decimal(7,2) (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

42448764 records
CPU times: user 3.43 ms, sys: 23 ms, total: 26.4 ms
Wall time: 31.3 s


### Limit number of records in dataframe

We can remove the category_id column, because it is a direct transformation of category_code (this could be turned into another table). It also helps make it easier to print the output here.

Next, we limit the dataframe to just a 5000-records subset. Notably, the dataframe is arranged by time, so this is how the subset will be biased.

In [4]:
df=df.drop("category_id")
df=df.limit(5000)
df.createOrReplaceTempView("records")

#### Print dataframe contents

In [5]:
%%time
spark.sql("SELECT * FROM records").show()

+-------------------+----------+----------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|furniture.living_...|    null| 543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|electronics.smart...|   apple|1081.98|535871217|c6bd7419-2748-4c5...|
|2019-10-01 00:00:05|      view|   1480613|   computers.desktop|  pulser| 908.62|512742880|0d0d9

#### Top prices among all event types

In [6]:
%%time
spark.sql("SELECT * FROM records ORDER BY price DESC").show(20)

+-------------------+----------+----------+--------------------+------+-------+---------+--------------------+
|         event_time|event_type|product_id|       category_code| brand|  price|  user_id|        user_session|
+-------------------+----------+----------+--------------------+------+-------+---------+--------------------+
|2019-10-01 02:26:24|      view|  21407288|  electronics.clocks|  rado|2574.07|527321544|b25dd047-7fec-489...|
|2019-10-01 02:26:42|      view|  29900054|                null|  peda|2557.59|555463012|ff140fa7-261d-495...|
|2019-10-01 02:23:25|      view|   1304849|  computers.notebook|  acer|2552.93|539527768|9df85aec-00be-464...|
|2019-10-01 02:23:40|      view|  21406787|  electronics.clocks|  null|2522.59|543091275|6a07dd91-2a76-48b...|
|2019-10-01 02:20:45|      view|   1305977|  computers.notebook| apple|2496.59|519578504|2a4700ff-7ff2-466...|
|2019-10-01 02:24:06|      view|   1306449|  computers.notebook|  acer|2293.24|539527768|9df85aec-00be-464...|
|

#### Who has bought how much

Note: Sum of prices purchased and number of purchases made are not directly correlated. This could lead to an interesting feature about multiple items being bought simultaneously vs. being bought one at a time over a long length of time.

In [7]:
%%time
spark.sql("SELECT COUNT(*) AS purchases_made, SUM(price), user_id FROM records WHERE event_type=='purchase' GROUP BY user_id ORDER BY COUNT(*) DESC").show(10,False)

+--------------+----------+---------+
|purchases_made|sum(price)|user_id  |
+--------------+----------+---------+
|3             |525.62    |555110488|
|2             |213.65    |517953667|
|2             |804.67    |551377651|
|2             |4.25      |514127317|
|2             |181.12    |549614436|
|1             |130.76    |543272936|
|1             |54.42     |555332717|
|1             |29.51     |514591159|
|1             |189.91    |524601178|
|1             |515.67    |524325294|
+--------------+----------+---------+
only showing top 10 rows

CPU times: user 1.79 ms, sys: 722 µs, total: 2.51 ms
Wall time: 2.36 s


#### Distribution of actions

In [8]:
%%time
spark.sql("SELECT COUNT(*), event_type FROM records GROUP BY event_type ORDER BY COUNT(*) DESC").show()

+--------+----------+
|count(1)|event_type|
+--------+----------+
|    4896|      view|
|      54|  purchase|
|      50|      cart|
+--------+----------+

CPU times: user 1.33 ms, sys: 1.01 ms, total: 2.33 ms
Wall time: 1.56 s
